# Function Decorators and Closures
Function decorators let us “mark” functions in the source code to enhance their behavior in some way. This is powerful stuff, but mastering it requires understanding closures.

One of the newest reserved keywords in Python is nonlocal, introduced in Python 3.0. You can have a profitable life as a Python programmer without ever using it if you adhere to a strict regimen of class-centered object orientation. However, if you want to implement your own function decorators, you must know closures inside out, and then the need for nonlocal becomes obvious.

In [1]:
# we can implement something like this
def double(x):
    return 2 * x

def add(a, b):
    return a + b

double(add(1, 2))

6

In [2]:
# as this
def double(func):
    def inner(*args, **kwargs):
        return 2 * func(*args, **kwargs)
    return inner

@double
def add(a, b):
    return a + b

add(1, 2)

6

In [3]:
add

<function __main__.double.<locals>.inner(*args, **kwargs)>

Remember to use functools.wraps for this issue.

In [4]:
import functools

def double(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        return 2 * func(*args, **kwargs)
    return inner

@double
def add(a, b):
    return a + b

add(1, 2)

6

In [5]:
add

<function __main__.add(a, b)>

## Decorators 101
A decorator is a callable that takes another function as argument (the decorated function).

In [6]:
def decorate(target_func):
    def inner():
        print("running inner()")
    return inner

@decorate
def target():
    print("running target()")

target()

running inner()


To summarize: the first crucial fact about decorators is that they have the power to replace the decorated function with a different one. The second crucial fact is that they are executed immediately when a module is loaded.

## When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined.

In [7]:
import sys
if "test_decorators" in sys.modules:
    del sys.modules["test_decorators"]

In [8]:
import test_decorators

running the function 'double'!


In [9]:
import sys
"test_decorators" in sys.modules

True

In [10]:
import inspect
lines = inspect.getsource(test_decorators)
print(lines)

import functools

def double(func):
    print("running the function 'double'!")
    @functools.wraps(func)
    def inner(*args, **kwargs):
        return 2 * func(*args, **kwargs)
    return inner

@double
def add(a, b):
    return a + b



function decorators are executed as soon as the module is imported, but the decorated functions only run when they are
explicitly invoked. This highlights the difference between what Pythonistas call import time and runtime.

## Decorator-Enhanced Strategy Pattern

In [11]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    ''' 5% discount for customers with 1000 or more fidelity points '''
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    ''' 10% discount for each LineItem with 20 or more units '''
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    '''7% discount for orders with 10 or more distinct items '''
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    ''' Select best discount available '''
    return max(promo(order) for promo in promos)



This example registers the function without changing it. Most decorators do change the decorated function. They usually do it by defining an inner function and returning it to replace the decorated function.  Code that uses inner functions almost always depends on closures to operate correctly. To understand closures, we need to take a step back a have a close look at how variable scopes work in Python.

## Variable Scope Rules

In [12]:
if "b" in globals():
    del b

def func(a):
    print(a)
    print(b)

try:
    func(1)
except Exception as e: # b is not defined
    print(e)

1
name 'b' is not defined


In [13]:
b = 2 # if we define b
func(1)

1
2


Now a more involved example.

In [14]:
import traceback

b = 2
def func(a):
    print(a)
    print(b)
    b = 3

try:
    func(1)
except:
    traceback.print_exc()

1


Traceback (most recent call last):
  File "<ipython-input-14-eb7c97220ba0>", line 10, in <module>
    func(1)
  File "<ipython-input-14-eb7c97220ba0>", line 6, in func
    print(b)
UnboundLocalError: local variable 'b' referenced before assignment


But the fact is, when Python compiles the body of the function, it decides that b is a local variable because it is assigned within the function. The generated bytecode reflects this decision and will try to fetch b from the local environment. Later, when the call func(1) is made, the body of f2 fetches and prints the value of the local variable a, but when
trying to fetch the value of local variable b it discovers that b is unbound.